Profile charts are weak and probably cant be easily added to the layout as a profile
Profile Scene is weak in terms of labeling and graphic display as a pipe profile tool

The best way to add an engineering profile to a layout is mathematically in a map

In [20]:
ws = r"\\citydata\users\kgonterwitz\projectreviews\westdale\WestdaleSewerDesignFile.gdb"

profile = "SewerProfile1"

projection = '''PROJCS["NAD_1983_2011_KS_RCS_Zone_11",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",11500000.0],PARAMETER["False_Northing",600000.0],PARAMETER["Central_Meridian",-95.25],PARAMETER["Standard_Parallel_1",39.1],PARAMETER["Scale_Factor",1.000033],PARAMETER["Latitude_Of_Origin",39.1],UNIT["Foot_US",0.3048006096012192]];-110278200 -95394100 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision'''

arcpy.env.overwriteOutput = 1

In [9]:
#select the linear infrastructure (pipe, street, etc) to profile
arcpy.conversion.FeatureClassToFeatureClass("ssGravityMain", ws, profile)


<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\SewerProfile1'>

In [10]:
#dissolve the selected lines into a single line that is the full profile length
#dissolve will keep 2 DS elevations for profiling drops

arcpy.management.Dissolve(ws+r"\\"+profile, ws+r"/"+profile+"D", "MAINTYPE", "UPELEV MAX;DOWNELEV MIN;DOWNELEV MAX;Shape_Length SUM", "MULTI_PART", "DISSOLVE_LINES", '')


<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\SewerProfile1D'>

In [11]:
#Profiles will be stationed based on upstream to downstream direction
#these steps derive coordinates for feature along the profile length
#XY coordinates are displayed as Stationing/ M domain coordiantes along the profile length
arcpy.management.CalculateField(ws+r"/"+profile+"D", "StartPoint", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.CreateRoutes(profile+"D", "MAINTYPE", ws+r"\\profileM", "TWO_FIELDS", "StartPoint", "SUM_Shape_Length", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.lr.LocateFeaturesAlongRoutes(profile, "profileM", "MAINTYPE", "0 Feet", ws+r"/profileSegments", "RID; Line; FMEAS; TMEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\profileSegments'>

In [23]:
#add manhole profiles
arcpy.lr.LocateFeaturesAlongRoutes("ssManhole", "profileM", "MAINTYPE", "0 Feet", ws+r"\ManholeM", "RID; Point; MEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")
arcpy.management.XYTableToPoint(ws+r"\ManholeM", ws+r"\ManholeProfileRimPt", "MEAS", "RIMELEV", None, projection)
arcpy.management.XYTableToPoint(ws+r"\ManholeM", ws+r"\ManholeProfileInvert", "MEAS", "INVERTELEV", None, projection)
arcpy.management.Merge("ManholeProfileInvert;ManholeProfileRimPt", ws+r"\ManholeProfileMerge", '#', "ADD_SOURCE_INFO")

arcpy.management.PointsToLine("ManholeProfileMerge", ws+r"\ManholeProfile", "FACILITYID", "INVERTELEV", "NO_CLOSE")
arcpy.management.AddJoin("ManholeProfile", "FACILITYID", "ManholeProfileInvert", "FACILITYID", "KEEP_ALL", "INDEX_JOIN_FIELDS")

arcpy.management.SelectLayerByAttribute("ManholeProfile", "NEW_SELECTION", "ManholeProfileInvert.BARRELDIA IS NULL", None)
arcpy.management.CalculateField("ManholeProfile", "BufferWidth", "2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute("ManholeProfile", "NEW_SELECTION", "ManholeProfileInvert.BARRELDIA IS NOT NULL", None)
arcpy.management.CalculateField("ManholeProfile", "BufferWidth", "!ManholeProfileInvert.BARRELDIA!/2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute("ManholeProfile", "CLEAR_SELECTION")

arcpy.analysis.Buffer("ManholeProfile", ws+r"\ManholeProfile_Buffer", "ManholeProfile.BufferWidth", "FULL", "FLAT", "NONE", None, "PLANAR")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\ManholeProfile_Buffer'>

In [12]:
#Now we should have the station coordinates and elevation of the pipe ends for the sewer
#this cell calculates the station and elevation for the profile lines as points
#then the points can be merged and rocessed into a profile line

arcpy.management.XYTableToPoint("profileSegments", ws+r"\ProfileSegStartPt", "FMEAS", "UPELEV", None, projection)
arcpy.management.XYTableToPoint("profileSegments", ws+r"\ProfileSegEndPt", "TMEAS", "DOWNELEV", None, projection)

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\ProfileSegEndPt'>

In [40]:
#the first example was an outside drop manhole, the downstream pipe elevation and slope are based on the low connection
#the slope and profile need to show the flowline of the upper manhole
#this will have to be entered manually for now
#drop manholes have a bottom drop of less than 2 ft per KDHE criteria
#the upper drop is to meet max slope and is higher than 2 ft

#arcpy.conversion.FeatureClassToFeatureClass("ManholeProfileInvert", ws, "DropMH", "MHTYPE = 'DRP'", '#', '')
arcpy.conversion.FeatureClassToFeatureClass("ManholeProfileInvert", ws, "BuriedMH", "MHTYPE = 'Buried'", '#', '')

DropElTop = 946.00
DropElTSta = 118.50799-6

DropElBottom = 945.00
DropElBSta = 118.50799

arcpy.management.CalculateField("BuriedMH", "FMEAS", "!MEAS!-2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("BuriedMH", "UPELEV", DropElBottom, "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("BuriedMH", "DNELEV", DropElBottom, "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.XYTableToPoint(ws+r"\BuriedMH", ws+r"\BuriedMH_1", "FMEAS", "DNELEV", None, projection)


<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\BuriedMH_1'>

In [41]:
#merge the points 
arcpy.management.Merge("ProfileSegStartPt;ProfileSegEndPt;BuriedMH_1", ws+r"\Point_Merge")
arcpy.management.Sort("Point_Merge", ws+r"\Point_Merge_Sort", "FMEAS ASCENDING;UPELEV ASCENDING", "UR")
#theck the points , may need to delete a point on a drop MH

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\Point_Merge_Sort'>

In [42]:
#merge the points and make the profile lines

arcpy.management.PointsToLine("Point_Merge_Sort", ws+r"\ProfleLine", "MAINTYPE", None, "NO_CLOSE")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\ProfleLine'>

In [32]:
#step through the range of elevations for the profile
topelevation = 956
baseelevation = 934
calcelevation =0
i=0

while calcelevation < topelevation:
    calcelevation = baseelevation+i*2
    arcpy.management.CalculateField(ws+r"\\"+profile+"D", "ProfileEl", calcelevation, "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
    arcpy.management.XYTableToPoint(ws+r"\\"+profile+"D", ws+r"\Profilea", "StartPoint", "ProfileEl", None, projection)
    arcpy.management.XYTableToPoint(ws+r"\\"+profile+"D", ws+r"\Profileb", "SUM_Shape_Length", "ProfileEl", None, projection)
    arcpy.management.Merge("Profilea;Profileb", ws+r"\\Point_Merge_scales")
    arcpy.management.PointsToLine(ws+r"\\Point_Merge_scales", ws+r"\\ProfleLine"+str(calcelevation), "MAINTYPE", None, "NO_CLOSE")
    arcpy.management.CalculateField(ws+r"\\ProfleLine"+str(calcelevation), "ProfileEl", calcelevation, "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
    print(i, calcelevation)
    i+=1
    



0 934
1 936
2 938
3 940
4 942
5 944
6 946
7 948
8 950
9 952
10 954
11 956


In [ ]:
#make a list of the outputs above and merge them to simplify labeling and symbology
arcpy.management.Merge(r"'ProfleLine956';'ProfleLine954';'ProfleLine952';'ProfleLine950';'ProfleLine948';'ProfleLine946';'ProfleLine944';'ProfleLine942';'ProfleLine940';'ProfleLine938';'ProfleLine936';'ProfleLine934'", ws+r"\ProfileLines_Merge", "#", "NO_SOURCE_INFO")

In [6]:
#create surface profile 
#DEM is used, consider using DSM

DEM = r"\\gisfile\gisdata\Published\LIDAR\2021_LiDAR\Lidar_Mosaic_Dataset.gdb\QL2_DEM_2021"
arcpy.ddd.InterpolateShape(DEM, ws+r"\\"+"profileM", ws+r"\SurfaceEl", None, 1, "BILINEAR", "DENSIFY", 0, "EXCLUDE")
arcpy.management.FeatureVerticesToPoints("SurfaceEl", ws+r"\SurfacePt", "ALL")
arcpy.management.AddXY("SurfacePt")
arcpy.lr.LocateFeaturesAlongRoutes("SurfacePt", "profileM", "MAINTYPE", "0 Feet", ws+r"\profileMZ", "RID; Point; MEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")
arcpy.management.XYTableToPoint("profileMZ", ws+r"\SurfaceMZPoint", "MEAS", "POINT_Z", None, projection)
arcpy.management.PointsToLine("SurfaceMZPoint", ws+r"\SurfaceMZ", "RID", "MEAS", "NO_CLOSE")

<Result 'C:\\Users\\kgonterwitz\\OneDrive - City of Lawrence KS\\Documents\\ArcGIS\\Projects\\KylesProProject3\\Profiles.gdb\\SurfaceMZ'>

In [8]:
#add the sewer lines as events for labeling the diameter, material, slope, etc

arcpy.management.CalculateField("ProfleLine", "StartM", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("ProfleLine", "EndM", "!Shape_Length!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.CreateRoutes("ProfleLine", "MAINTYPE", ws+r"\ProfleLineRoute", "TWO_FIELDS", "StartM", "EndM", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.lr.MakeRouteEventLayer("ProfleLineRoute", "MAINTYPE", "profileSegments", "Main Type; Line; FMEAS; TMEAS", "profileSegmentsEvents", None, "NO_ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")


<Result 'ManholeProfile'>